In [26]:
import pandas as pd
import csv
import config

In [27]:
pureDF = pd.DataFrame(columns=["Date", "Amount", "Description", "Additional", "Category"])

In [28]:
for f in config.IMPORT_FILES:
    print ("Loading File: ", f)
    df = pd.read_csv(f, header=None)

    if df.columns.size == 13:
        # Reset header to match line 1
        new_header = df.iloc[0]
        df = df[1:]
        df.columns = new_header

        # Filter rows
        df = df[df["Card Member"] == config.CARD_MEMBER_FILTER]

        # Transform data types
        df['Date'] = pd.to_datetime(df['Date'])
        df['Amount'] = pd.to_numeric(df['Amount']) * -1.0
        df['Additional'] = df["Extended Details"]

        # Remove Unused Columns
        df = df.drop("Address", axis=1)
        df = df.drop("City/State", axis=1)
        df = df.drop("Country", axis=1)
        df = df.drop("Zip Code", axis=1)
        df = df.drop("Card Member", axis=1)
        df = df.drop("Account #", axis=1)
        df = df.drop("Reference", axis=1)
        df = df.drop("Extended Details", axis=1)
        df = df.drop("Appears On Your Statement As", axis=1)

    elif df.columns.size == 5:
        df.rename(columns = {0: 'Date', 1: 'Amount', 2: "Unused", 3: "Additional", 4: 'Description'}, inplace=True)
        df['Date'] = pd.to_datetime(df['Date'])
        df = df.drop("Unused", axis=1)

    else:
        raise("Unknown Data Type!  Columns=" + str(df.columns.size))    

    pureDF = pd.concat([pureDF, df], axis=0)
    print("Added", str(len(df)), "records to data set, resulting in a new total of", str(len(pureDF)))
    print()

Loading File:  ~/Downloads/activity.csv
Added 1068 records to data set, resulting in a new total of 1068

Loading File:  ~/Downloads/activity (1).csv
Added 514 records to data set, resulting in a new total of 1582

Loading File:  ~/Downloads/Checking2.csv
Added 1136 records to data set, resulting in a new total of 2718



/var/folders/pp/350d28rx0g1dmhq658jwgy8c0000gn/T/ipykernel_48733/2820223699.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pureDF = pd.concat([pureDF, df], axis=0)


In [29]:
print(pureDF.columns)
print(pureDF.shape)
pureDF.head()

Index(['Date', 'Amount', 'Description', 'Additional', 'Category'], dtype='object')
(2718, 5)


,Date,Amount,Description,Additional,Category
1,2023-12-31,-27.10,BRUSTERS ICE CREAM #WOODSTOCK GA,74744554001 770-720-2868\nBRUSTERS ICE CREAM #...,Restaurant-Bar & Café
2,2023-12-30,-5.45,LEVY MERCEDES BENZ SATLANTA GA,73011933365 JSCHERGER@LEVYRESTAUR\nJSCHERGER@L...,Restaurant-Restaurant
3,2023-12-30,-16.34,LEVY MERCEDES BENZ SATLANTA GA,73011933365 JSCHERGER@LEVYRESTAUR\nJSCHERGER@L...,Restaurant-Restaurant
4,2023-12-30,-67.61,MAPLE STREET BISCUITALPHARETTA GA,99999993364 6152354040\nMAPLE STREET BISCUIT C...,Restaurant-Bar & Café
5,2023-12-30,-57.47,QT 765/QUIKTRIP 5542ALPHARETTA GA,0076502TWYH OUTSIDE SALE\nQT 765/QUIKTRIP 5542...,Transportation-Fuel


In [30]:
pureDF.to_csv(config.OUTPUT_FILE, index=False, quoting=csv.QUOTE_ALL)